<a href="https://colab.research.google.com/github/Dev-jalexx/Pytorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Convert the MNIST Image files into 4-dimensions Tensors(Number of images, Height, Width, Color Channel)
transform = transforms.ToTensor()

In [4]:
# Train data
train_data = datasets.MNIST(root='/cnn_data', download = True, train= True, transform = transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 486kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.56MB/s]


In [5]:
# Test data
test_data = datasets.MNIST(root='/cnn_data', download = True, train= False, transform = transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [19]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [44]:
# define our CNN model
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [21]:
#Grab 1 MNIST image
for i, (X_train, y_train) in enumerate(train_data):
    break

In [23]:
X_train.shape

torch.Size([1, 28, 28])

In [45]:
# Transform into 4-dimension
x = X_train.view(1, 1, 28, 28)

In [46]:
# first convolutional layer
x = F.relu(conv1(x))

In [47]:
x.shape # Padding reduces the dimensions

torch.Size([1, 6, 26, 26])

In [48]:
# first pooling
x = F.max_pool2d(x, 2, 2) # kernel size of 2 and stride of 2

In [49]:
x.shape # 26 / 2 = 13

torch.Size([1, 6, 13, 13])

In [40]:
# Second convolutional layer
x = F.relu(conv2(x))

In [41]:
x.shape

torch.Size([1, 16, 11, 11])

In [42]:
# Second pooling
x = F.max_pool2d(x, 2, 2)

In [43]:
x.shape

torch.Size([1, 16, 5, 5])

In [53]:
# Model
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)

    # Fully connected Layers
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2)
    #Second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2)
    # Review to flatten it out/ reshape the CNN tensor from 4D to 2D
    X = X.view(-1, 16*5*5) # -1 preserves the original batch size.
    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1) # Convets output into log-probablities

In [54]:
# Instantiate our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [55]:
# Losss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [60]:
import time
start_time = time.time()
# Create variables for tracking
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For loops of epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  #Train
  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    predicted = torch.max(y_pred.data, 1)[1] #Add up number of correct predictions
    batch_corr = (predicted == y_train).sum() # How many we got correct
    trn_corr += batch_corr

    #Update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #Print results
    if b%600 == 0:
      print(f'epoch: {i} batch: {b} loss: {loss.item()}')
  train_losses.append(loss)
  train_correct.append(trn_corr)

  #Test
  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1]# Add up correct predictions
      tst_corr += (predicted == y_test).sum()# T=1, F=0
  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)



current_time  = time.time()
total = current_time - start_time
print(f'Training took {total/60} minutes.')

epoch: 0 batch: 600 loss: 0.06673242151737213
epoch: 0 batch: 1200 loss: 0.011486882343888283
epoch: 0 batch: 1800 loss: 0.00010850733087863773
epoch: 0 batch: 2400 loss: 0.0034949115943163633
epoch: 0 batch: 3000 loss: 0.5454784631729126
epoch: 0 batch: 3600 loss: 0.0006972571136429906
epoch: 0 batch: 4200 loss: 0.011702567338943481
epoch: 0 batch: 4800 loss: 0.009425041265785694
epoch: 0 batch: 5400 loss: 0.003721428569406271
epoch: 0 batch: 6000 loss: 0.0005819665966555476
epoch: 1 batch: 600 loss: 0.000231283760513179
epoch: 1 batch: 1200 loss: 0.03065582737326622
epoch: 1 batch: 1800 loss: 0.0007656434318050742
epoch: 1 batch: 2400 loss: 0.04615493491292
epoch: 1 batch: 3000 loss: 0.0006118264282122254
epoch: 1 batch: 3600 loss: 0.0005625953199341893
epoch: 1 batch: 4200 loss: 0.0005126988398842514
epoch: 1 batch: 4800 loss: 0.033976007252931595
epoch: 1 batch: 5400 loss: 0.00010771364031825215
epoch: 1 batch: 6000 loss: 0.0075939493253827095
epoch: 2 batch: 600 loss: 2.8499747713